# Anime Analysis 

#### Source: https://www.anime-planet.com/ 
##### The best place to find new anime to enjoy!

#### Before we can start, we need to import the required packages: 

In [1]:
import sys
import re
import requests
from bs4 import BeautifulSoup
import time
from multiprocessing import Pool
import pandas as pd

In [2]:
exist = pd.read_pickle("anime_df.pkl")

In [3]:
sys.setrecursionlimit(0x100000)

#### Then we can scrape the data from the website: 

In [4]:
r = requests.get('https://www.anime-planet.com/anime/all?page=1&bvm=list')
soup = BeautifulSoup(r.text, 'lxml')

#### Let's take a look at what we got: 

In [5]:
soup

<!DOCTYPE html>
<html class="no-js" lang="en" xml:lang="en">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# profile: http://ogp.me/ns/profile# book: http://ogp.me/ns/book# video: http://ogp.me/ns/video#">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Browse All Anime | Anime-Planet</title>
<meta content="Create of a list of anime you've seen, watch them online, discover new anime and more on Anime-Planet. Search thousands of anime by your favorite tags and genres, studios, years, ratings, and more!" name="description"/>
<meta content="" name="keywords"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" name="viewport"/>
<link href="/apple-touch-icon.png?v=WGowMEAKpM" rel="apple-touch-icon"/>
<link href="/apple-touch-icon.png?v=WGowMEAKpM" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png?v=WGowMEAKpM" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png?v=WGowMEA

#### Crap... what have I gotten myself into?..   
#### Oh wait.. Is it a bird? Is it a plane? No, it's Beautiful Soup! To the rescue!

In [28]:
title, media_format, url = ([] for i in range(3))

In [29]:
def scraper(i):
    
    print(i)
    #global title, media_format, url
    
    page = 'https://www.anime-planet.com/anime/all?page=' + str(i) + '&bvm=list'
    r = requests.get(page)
    soup = BeautifulSoup(r.text, 'lxml')
    
    #if soup.find_all('td', attrs={'class': "tableTitle"}) not in exist['Title']:
    
    if soup.find_all('td', attrs={'class': "tableTitle"}):
        #title += soup.find_all('td', attrs={'class': "tableTitle"})
        title = soup.find_all('td', attrs={'class': "tableTitle"})
    else:
        title = 'na'



    if soup.find_all('td', attrs={'class': "tableType"}):
        #media_format += soup.find_all('td', attrs={'class': "tableType"})
        media_format = soup.find_all('td', attrs={'class': "tableType"})
    else:
        #media_format.append('na')
        media_format = 'na'

    url = []
    for section in soup.find_all('td', attrs={'class': "tableTitle"}): 
        extract = BeautifulSoup(str(section))
        if extract.find('a', {'data-tooltip-entry':'anime'})['href']:
            mini = 'https://www.anime-planet.com' + extract.find('a', {'data-tooltip-entry':'anime'})['href']
            url.append(mini)
        else:
            url.append('na')

    return [title, media_format, url]
    
    


In [30]:
p = Pool()
title, media_format, url = zip(*p.map(scraper, range(1,368)))
p.terminate()
p.join()

13
37
1
25
85
49
61
73
2
38
62
50
14
86
26
74
3
39
63
15
51
87
27
75
4
40
16
64
52
88
28
76
5
41
53
65
17
89
29
77
6
42
54
18
66
30
90
78
7
43
55
19
67
31
8
91
79
44
20
68
32
56
9
92
80
45
21
57
33
69
10
93
81
46
22
34
58
70
11
94
82
47
23
59
35
12
71
83
24
95
48
60
36
72
97
84
96
109
98
121
133
110
99
145
122
157
111
134
169
100
146
123
158
112
135
170
101
147
124
159
113
136
102
125
171
148
160
114
137
103
126
172
149
161
115
104
173
138
127
150
162
116
105
139
174
128
117
163
106
151
140
175
129
118
107
164
152
141
176
130
119
165
108
142
177
153
131
120
166
143
154
178
132
167
181
179
144
155
193
205
182
168
217
180
156
194
229
206
183
218
241
230
195
253
184
207
265
219
242
231
196
254
208
185
266
220
243
232
197
255
209
186
221
267
244
233
198
256
210
187
222
268
234
245
199
257
211
188
223
269
235
246
212
200
189
224
258
270
236
247
213
201
190
225
271
259
237
214
248
191
202
226
272
260
238
215
203
192
249
227
273
261
239
216
204
277
250
228
274
262
240
289
278
251
301
275
263


In [34]:
title[0][0].text

'Fullmetal Alchemist: Brotherhood'

In [35]:
tmp_title = title
tmp_format = media_format
tmp_url = url
title, media_format, url = ([] for i in range(3))
for i in range(0, len(tmp_title)):
    for j in range(0, len(tmp_title[i])):
        if (tmp_title[i][j]).text not in exist['Title'].values:
            title.append(tmp_title[i][j])
            media_format.append(tmp_format[i][j])
            url.append(tmp_url[i][j])

In [36]:
for i in range(len(title)):
    title[i] = title[i].text
    media_format[i] = media_format[i].text

In [37]:
title[0:10]

["One Piece 3D2Y: Overcoming Ace's Death! Luffy's Pledge to His Friends!",
 'Wise Man’s Grandchild',
 'Tsubasa: Spring Thunder Chronicle',
 "Gurren Lagann Movie 1: Childhood's End",
 "xxxHOLiC Movie: A Midsummer Night's Dream",
 'The Heroic Legend of Arslan: Dust Storm Dance',
 'You and Me. 2',
 'Attack on Titan 3rd Season Specials',
 'xxxHOLiC',
 'Kakegurui XX']

In [38]:
len(url)

267

In [39]:
alt, eps, studio, year_range, desc, tags, acc_rating, votes = ([] for i in range(8))

In [40]:
count = 1
def get_each(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    global count
    print(count)
    count += 1

    if soup.find_all('h2', attrs={'class': "aka"}):
        #alt += soup.find_all('h2', attrs={'class': "aka"})
        alt = (soup.find_all('h2', attrs={'class': "aka"})[0].text)[12:-1]
    else:
        #alt.append('na')
        alt = 'na'
        
    if soup.find_all('span', attrs={'class': "type"}):
        #eps += soup.find_all('span', attrs={'class': "type"})
        eps = soup.find_all('span', attrs={'class': "type"})[0].text
    else:
        #eps.append('na')
        eps = 'na'
        
    studio = []
    if soup.find_all('a', {"href" : lambda L: L and L.startswith('/anime/studios/')}):
        #stds = []
        #stds = soup.find_all('a', {"href" : lambda L: L and L.startswith('/anime/studios/')})
        stds = soup.find_all('a', attrs={"href" : lambda L: L and L.startswith('/anime/studios/')})
        
        for i in range(0, len(stds)):
            studio.append(stds[i].text)
            
        '''if len(stds) > 1:
            #studio.append(stds)
            studio = stds
            print(studio)
        else:
            #studio += stds
            studio = stds
            print(studio)'''
    else:
        studio = 'na'
        
    if soup.find_all('span', attrs={'class': "iconYear"}):
        #year_range += soup.find_all('span', attrs={'class': "iconYear"})
        year_range = soup.find_all('span', attrs={'class': "iconYear"})[0].text
    else:
        #year_range.append('na')
        year_range = 'na'
        
    
    if soup.find_all('div', attrs={'itemprop': 'description'}):
        #desc += soup.find_all('div', attrs={'itemprop': 'description'})
        desc = soup.find_all('div', attrs={'itemprop': 'description'})[0].text
    else:
        #desc.append('na')
        desc = 'na'
    
    if soup.find_all('a', attrs={'data-tooltip': 'tags'}):
        #tags.append(soup.find_all('a', attrs={'data-tooltip': 'tags'}))
        tags = soup.find_all('a', attrs={'data-tooltip': 'tags'})
        
        for i in range(0, len(tags)):
            tags[i] = (tags[i].text)[1:-1]
        
    else:
        #tags.append('na')
        tags = 'na'
        
    acc_rating, votes = ([] for i in range(2))
    
    for section in soup.find_all('div', attrs={'class': "avgRating"}):
        extract = BeautifulSoup(str(section))
        
        if extract.find('meta', {'itemprop':'ratingValue'}):
            #acc_rating.append(extract.find('meta', {'itemprop':'ratingValue'})['content'])
            acc_rating = float(extract.find('meta', {'itemprop':'ratingValue'})['content'])
        else:
            acc_rating.append('na')                            

        if extract.find('meta', {'itemprop':'ratingCount'}):
            #votes.append(extract.find('meta', {'itemprop':'ratingCount'})['content'])
            votes = int(extract.find('meta', {'itemprop':'ratingCount'})['content'])
        else:
            votes.append('na')
            
    return [alt, eps, studio, year_range, desc, tags, acc_rating, votes ]

In [41]:
get_each('https://www.anime-planet.com/anime/one-piece-special-protect-the-last-great-stage')

1


['One Piece Special 3: Mamore! Saigo no Dai Butai',
 'TV Special (1 ep x 46 min)',
 ['Toei Animation'],
 ' 2003 ',
 'The renowned Randolph troupe is in town and they are putting on a special farewell play to mark their leader’s retirement. When three of its actors quit a mere thirty minutes before the curtain rises, the Straw Pirates offer their services; though with Robyn’s overacting, Usopp trying to hog the limelight and Luffy gate-crashing the stage dressed as a monkey, the first act becomes more of a farce instead! Before the second act can commence, Randolph is arrested by the marines for selling stolen weapons to pirates, and boards the ship to prove his innocence. There, he’s confronted by the Marine Commander named Governor, a former subordinate of Randolph’s, who’s concocted the whole plan in order to exact his revenge on the ex-marine. Now, in addition to rescuing Randolph and protecting his ship, Luffy must also convince the troupe leader that not all pirates are bloodthirs

In [42]:
p = Pool()
alt, eps, studio, year_range, desc, tags, acc_rating, votes  = zip(*p.map(get_each, url))
p.terminate()
p.join()

2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
9
8
9
9
9
9
9
9
10
9
10
10
10
10
10
10
11
10
11
11
11
11
12
11
12
11
12
11
12
12
12
13
12
13
13
12
13
13
13
13
14
14
14
13
14
14
14
14
15
15
15
14
15
15
15
16
15
16
16
16
15
16
16
17
17
16
17
17
16
17
17
18
18
17
18
18
17
18
18
19
19
18
19
19
18
19
20
20
20
19
20
19
19
21
21
21
20
20
20
20
21
22
22
21
21
22
22
21
21
22
23
22
23
22
23
23
22
24
23
23
24
23
24
23
24
24
25
24
25
24
24
25
25
25
25
25
26
26
25
26
26
27
26
26
27
26
27
26
27
27
28
28
27
27
27
28
28
29
28
28
29
29
29
28
28
30
29
30
29
30
30
31
30
30
31
31
31
31
31
32
32
32
32
32
33
33
33
32
33
34
33
33
34
34
34
34
35
35
34
35
35
35
36
36
36
36
36
37
37
37
37
37


In [43]:
alt = list(alt)
eps = list(eps)
studio = list(studio)
year_range = list(year_range)
desc = list(desc)
tags = list(tags)
acc_rating = list(acc_rating)
votes = list(votes)

In [44]:
year_range[-10:-1]

[' 2019 - ? ',
 ' 2019 ',
 ' 2019 - ? ',
 ' 2019 ',
 ' 2018 ',
 ' 2019 - ? ',
 ' 2019 ',
 ' 2019 ',
 ' 2019 ']

In [45]:
print(acc_rating[0])
print(acc_rating[-1])

8.612
['na']


In [46]:
len(acc_rating)

267

In [47]:
dur, comp = ([] for i in range(2))

for i in range(len(eps)):
    nums = re.findall("(\d+)", str(eps[i]))
    
    if eps[i].find('+') != -1 or year_range[i].find('TBA') != -1 or year_range[i].find('?') != -1:
        comp.append(0)
    else:
        comp.append(1)
        
    if len(nums) == 1:
        eps[i] = int(nums[0])
        dur.append('na')
    elif len(nums) > 1:
        eps[i] = int(nums[0])
        dur.append(int(nums[1]))
    else:
        eps[i] = 'na'
        dur.append('na')
            
            

In [48]:
df = pd.DataFrame({'Rating': acc_rating, 'Votes': votes, 'Title':title, 'Alt Title':alt, 'Format':media_format, 'Episodes':eps, 'Duration':dur, 'Completed':comp, 'Studio':studio, 'Year':year_range, 'Tags':tags, 'Description':desc})

In [49]:
df

,Rating,Votes,Title,Alt Title,Format,Episodes,Duration,Completed,Studio,Year,Tags,Description
0,8.612,3473,One Piece 3D2Y: Overcoming Ace's Death! Luffy'...,"One Piece: ""3D2Y"", One Piece 3D2Y: Ace no shi...",TV Special,1,107,1,[Toei Animation],2014,"[Action, Fantasy, Shounen, Pirates, Superpower...","While training under Rayleigh, Luffy must lear..."
1,8.606,1194,Wise Man’s Grandchild,Kenja no Mago,TV,3,na,0,[SILVER LINK],2019 - ?,"[Action, Adventure\n, Fantasy, Magic, Overpowe...",A young man dies in a car accident and is rebo...
2,8.58,3720,Tsubasa: Spring Thunder Chronicle,Tsubasa: Shunraiki,OVA,2,na,1,[Production I.G],2009,"[Adventure, Fantasy, Shounen, Fated Lovers, Ma...","Trapped in the ice world of Celes, the struggl..."
3,8.568,7357,Gurren Lagann Movie 1: Childhood's End,Tengen Toppa Gurren Lagann Movie 1: Guren-hen,Movie,1,120,1,[Gainax],2008,"[Action, Adventure, Comedy, Mecha, Sci Fi, Com...",Simon lives a boring life in the underground v...
4,8.468,6151,xxxHOLiC Movie: A Midsummer Night's Dream,xxxHOLiC Movie: Manatsu no Yoru no Yume,Movie,1,60,1,[Production I.G],2005,"[Seinen, Japanese Mythology, Supernatural, Bas...",Watanuki faces challenges in every area. Spiri...
5,8.422,3995,The Heroic Legend of Arslan: Dust Storm Dance,Arslan Senki: Fuujin Ranbu,TV,8,na,1,[LIDEN FILMS],2016,"[Action, Adventure, Fantasy, Shounen, Medieval...",The Lusitania forces continue to hold rule ove...
6,8.386,2555,You and Me. 2,Kimi to Boku. 2,TV,13,na,1,[J.C. Staff],2012,"[Comedy, Shounen, Slice of Life, Iyashikei, Sc...","""No matter how many years go by, I'm sure we'l..."
7,8.36,34,Attack on Titan 3rd Season Specials,Shingeki no Kyojin 3rd Season Specials,DVD Special,4,2,1,na,2019,"[Comedy, Fantasy, Shounen, Chibi, Parody, Base...",na
8,8.31,16465,xxxHOLiC,na,TV,24,na,1,[Production I.G],2006,"[Comedy, Seinen, Episodic, Japanese Mythology,...",Kimihiro Watanuki is cursed with the ability t...
9,8.262,1826,Kakegurui XX,na,TV,12,na,1,[Mappa],2019,"[Shounen, Gambling, High Stakes Games, Mind Ga...",Second season of Kakegurui.


In [50]:
df.loc[0,'Tags'][0]

'Action'

In [65]:
#df.to_csv("anime_list.csv", sep=';', index=False)

In [51]:
df.to_csv("anime_list.csv", sep=';', index=False, mode='a')

In [286]:
p = Pool(10)
p.map(scrape_indiv, url[0:100])
p.terminate()
p.join()

Process ForkPoolWorker-303:
Process ForkPoolWorker-300:
Process ForkPoolWorker-304:
Process ForkPoolWorker-302:
Process ForkPoolWorker-305:
Process ForkPoolWorker-299:
Process ForkPoolWorker-297:
Process ForkPoolWorker-298:
Process ForkPoolWorker-301:
Process ForkPoolWorker-296:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
 

KeyboardInterrupt: 

  File "/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/anaconda3